<a href="https://colab.research.google.com/github/detektor777/colab_list_video/blob/main/vid_stab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture
import os
import subprocess

subprocess.run(["apt", "update"], check=True)
subprocess.run(["apt", "install", "-y", "cmake", "git", "build-essential"], check=True)

vidstab_dir = "/content/vid.stab"
if not os.path.exists(vidstab_dir):
    subprocess.run(["git", "clone", "https://github.com/georgmartius/vid.stab.git", vidstab_dir], check=True)
else:
    print(f"Directory {vidstab_dir} already exists, skipping cloning.")

os.chdir(vidstab_dir)
subprocess.run(["cmake", "-DCMAKE_INSTALL_PREFIX=/usr/local"], check=True)
subprocess.run(["make"], check=True)
subprocess.run(["make", "install"], check=True)

subprocess.run(["ldconfig"], check=True)
os.environ["LD_LIBRARY_PATH"] = "/usr/local/lib:" + os.environ.get("LD_LIBRARY_PATH", "")

!ffmpeg -codecs | grep libx264


In [ ]:
#@title ##**Select Video File** { display-mode: "form" }
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
from google.colab import drive

upload_option = "Load from Google Drive Root"  #@param ["Upload from PC", "Load from Google Drive Root", "Load from Google Drive"]

file_name = None
last_selected_button = None

def reset_button_colors(buttons):
    for btn in buttons:
        btn.style.button_color = None

if upload_option == "Upload from PC":
    print("Please upload a video file.")
    uploaded = files.upload()
    if uploaded:
        file_name = list(uploaded.keys())[0]
    else:
        print("No file uploaded.")
        file_name = None

elif upload_option == "Load from Google Drive Root":
    drive.mount('/content/drive')
    root_dir = '/content/drive/MyDrive/'

    video_extensions = ['.mp4', '.mkv', '.avi', '.mov']
    files_list = []

    for f in os.listdir(root_dir):
        if os.path.isfile(os.path.join(root_dir, f)) and os.path.splitext(f)[1].lower() in video_extensions:
            files_list.append(f)

    if not files_list:
        print("No video files found in Google Drive root.")
        file_name = None
    else:
        print("Select a video file from Google Drive root:")

        output = widgets.Output()
        buttons = []

        def on_button_clicked(b):
            global file_name, last_selected_button
            with output:
                clear_output()
                reset_button_colors(buttons)
                selected_file = b.description
                file_name = os.path.join(root_dir, selected_file)

                if file_name and os.path.exists(file_name):
                    b.style.button_color = 'green'
                else:
                    b.style.button_color = 'red'

                last_selected_button = b
                print(f"Selected file: {file_name if file_name else 'None'}")

        for file in files_list:
            button = widgets.Button(description=file, layout=widgets.Layout(width='500px', overflow='hidden', text_overflow='ellipsis'))
            button.on_click(on_button_clicked)
            buttons.append(button)

        display(widgets.VBox(buttons), output)

elif upload_option == "Load from Google Drive":
    drive.mount('/content/drive')
    root_dir = '/content/drive/MyDrive/'

    video_extensions = ['.mp4', '.mkv', '.avi', '.mov']
    files_list = []

    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            if os.path.splitext(f)[1].lower() in video_extensions:
                relative_path = os.path.relpath(os.path.join(dirpath, f), root_dir)
                files_list.append(relative_path)

    if not files_list:
        print("No video files found in Google Drive or its subfolders.")
        file_name = None
    else:
        print("Select a video file from Google Drive (including subfolders):")

        output = widgets.Output()
        buttons = []

        def on_button_clicked(b):
            global file_name, last_selected_button
            with output:
                clear_output()
                reset_button_colors(buttons)
                selected_file = b.description
                file_name = os.path.join(root_dir, selected_file)

                if file_name and os.path.exists(file_name):
                    b.style.button_color = 'green'
                else:
                    b.style.button_color = 'red'

                last_selected_button = b
                print(f"Selected file: {file_name if file_name else 'None'}")

        for file in files_list:
            button = widgets.Button(description=file, layout=widgets.Layout(width='500px', overflow='hidden', text_overflow='ellipsis'))
            button.on_click(on_button_clicked)
            buttons.append(button)

        display(widgets.VBox(buttons), output)

if file_name:
    print(f"Video file path set to: {file_name}")
else:
    print("Video file path not set. Please select a file.")

In [ ]:
#@title ##**Config** { display-mode: "form" }
import subprocess
import os
from google.colab import files
import json

output_folder = "root" #@param ["google_drive","root"]

use_defaults = False  #@param {type:"boolean"}

shakiness = 5  #@param {type:"slider", min:1, max:10, step:1}
accuracy = 15  #@param {type:"slider", min:1, max:15, step:1}
stepsize = 6   #@param {type:"slider", min:1, max:32, step:1}
mincontrast = 0.3  #@param {type:"slider", min:0.0, max:1.0, step:0.1}

smoothing = 4  #@param {type:"slider", min:0, max:30, step:1}
optalgo = "gauss"  #@param ["gauss", "avg"]
maxshift = 30  #@param {type:"integer"}
maxangle = 0.05  #@param {type:"number"}
crop = "keep"  #@param ["keep", "black"]
zoom = 0  #@param {type:"slider", min:-50, max:50, step:1}
optzoom = 1  #@param {type:"slider", min:0, max:2, step:1}
zoomspeed = 0.25  #@param {type:"slider", min:0.0, max:5.0, step:0.25}
interpol = "bilinear"  #@param ["no", "linear", "bilinear", "bicubic"]

In [ ]:
#@title ##**Run** { display-mode: "form" }
import subprocess
import os
from google.colab import files
import json

defaults = {
    "vidstabdetect": {
        "shakiness": 5,
        "accuracy": 15,
        "stepsize": 6,
        "mincontrast": 0.3
    },
    "vidstabtransform": {
        "smoothing": 10,
        "optalgo": "gauss",
        "maxshift": -1,
        "maxangle": -1,
        "crop": "keep",
        "zoom": 0,
        "optzoom": 1,
        "zoomspeed": 0.25,
        "interpol": "bilinear"
    }
}

if 'file_name' not in globals() or file_name is None or not os.path.exists(file_name):
    print("Error: Video file not selected or doesn't exist.")
else:
    print(f"Stabilizing video: {file_name}")

    base_file_name = os.path.basename(file_name)
    output_file_name = base_file_name.rsplit('.', 1)[0] + '_stabilized.mp4'
    if output_folder == "google_drive":
        save_path = '/content/drive/MyDrive/'
    else:
        save_path = '/content/'
    output_video = os.path.join(save_path, output_file_name)

    transforms_file = "/content/transforms.trf"

    if use_defaults:
        detect_params = defaults["vidstabdetect"]
        transform_params = defaults["vidstabtransform"]
        print("Using default vid.stab parameters:")
    else:
        detect_params = {
            "shakiness": shakiness,
            "accuracy": accuracy,
            "stepsize": stepsize,
            "mincontrast": mincontrast
        }
        transform_params = {
            "smoothing": smoothing,
            "optalgo": optalgo,
            "maxshift": maxshift,
            "maxangle": maxangle,
            "crop": crop,
            "zoom": zoom,
            "optzoom": optzoom,
            "zoomspeed": zoomspeed,
            "interpol": interpol
        }
        print("Using custom settings from form.")

    print("Running first pass (vidstabdetect)...")
    cmd_detect = [
        "ffmpeg",
        "-i", file_name,
        "-vf", f"vidstabdetect=result={transforms_file}:shakiness={detect_params['shakiness']}:accuracy={detect_params['accuracy']}:stepsize={detect_params['stepsize']}:mincontrast={detect_params['mincontrast']}",
        "-f", "null",
        "-"
    ]
    result_detect = subprocess.run(cmd_detect, capture_output=True, text=True)

    print("Getting video metadata...")
    cmd_probe = [
        "ffprobe",
        "-v", "error",
        "-select_streams", "v:0",
        "-show_entries", "stream=codec_name,codec_type,bit_rate",
        "-of", "json",
        file_name
    ]
    result_probe = subprocess.run(cmd_probe, capture_output=True, text=True)
    video_info = json.loads(result_probe.stdout)
    video_stream = video_info['streams'][0]
    codec_name = video_stream.get('codec_name', 'libx264')
    bit_rate = video_stream.get('bit_rate', '5000000')

    print("Running second pass (vidstabtransform)...")
    cmd_transform = [
        "ffmpeg",
        "-i", file_name,
        "-vf", f"vidstabtransform=input={transforms_file}:smoothing={transform_params['smoothing']}:optalgo={transform_params['optalgo']}:maxshift={transform_params['maxshift']}:maxangle={transform_params['maxangle']}:crop={transform_params['crop']}:zoom={transform_params['zoom']}:optzoom={transform_params['optzoom']}:zoomspeed={transform_params['zoomspeed']}:interpol={transform_params['interpol']},unsharp=5:5:0.8:3:3:0.4",
        "-c:v", codec_name,
        "-b:v", bit_rate,
        "-map", "0",
        "-c:a", "copy",
        "-c:s", "copy",
        "-preset", "slow",
        "-y",
        output_video
    ]
    result_transform = subprocess.run(cmd_transform, capture_output=True, text=True)


In [ ]:
#@title ##**Download** { display-mode: "form" }
import os
from google.colab import files

if os.path.exists(output_video):
    print(f"Stabilized video saved at: {output_video}")
    files.download(output_video)
else:
    print("Error: Stabilized video was not created.")
